In [1]:
import logging
import pickle
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Things to include:
* bias initialization with 1 +
* dropout (recurrent?) + https://github.com/tensorflow/tensorflow/blob/v1.3.0/tensorflow/contrib/keras/python/keras/layers/recurrent.py#L140 
* Adam / SGD with learning rate annealing + momentum
* self-attention
* Bayesian Optimization for hyperparameters
* Gradient norm clipping +

In [2]:
MODEL_PATH = "../models/"

In [3]:
DATA_PATH = "../data/"
X_train = pickle.load(open(DATA_PATH + "X_train.p", "rb"))
X_dev = pickle.load(open(DATA_PATH + "X_dev.p", "rb"))
y_train = pickle.load(open(DATA_PATH + "y_train.p", "rb"))
y_dev = pickle.load(open(DATA_PATH + "y_dev.p", "rb"))

In [4]:
y_train.shape

(111699, 6)

In [5]:
train_text = X_train['comment_text']

In [6]:
tokenizer = Tokenizer(num_words=18400)
tokenizer.fit_on_texts(train_text)
sequences = tokenizer.texts_to_sequences(train_text)

In [8]:
word_index = tokenizer.word_index
embeddings_index = {}
f = open('../../../embeddings/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [9]:
print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(embedding_matrix.shape[0],
                            300,
                            weights=[embedding_matrix],
                            input_length=200, trainable = False,  mask_zero=True)

Found 400000 word vectors.


In [10]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_dev_texts = tokenizer.texts_to_sequences(X_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')

In [11]:
# class_weight = class_weight.compute_class_weight('balanced', np.unique(current_y_train), current_y_train)
# class_weight = {i:x for i, x in enumerate(list(class_weight))}
# class_weight

In [12]:
adam = keras.optimizers.Adam(clipnorm=1.)

In [16]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm = keras.layers.LSTM(50, return_sequences=True, unit_forget_bias = True, recurrent_dropout = 0.2, dropout = 0.5,  kernel_initializer='he_normal')(embedded_sequences)
lstm2 = keras.layers.LSTM(10, unit_forget_bias = True, recurrent_dropout = 0.2, dropout = 0.5,  kernel_initializer='he_normal')(lstm)
x = Dense(6, activation='sigmoid')(lstm2)

model = Model(sequence_input, x)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          50649300  
_________________________________________________________________
lstm_5 (LSTM)                (None, 200, 50)           70200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
Total params: 50,722,006
Trainable params: 72,706
Non-trainable params: 50,649,300
_________________________________________________________________


In [ ]:
model.fit(x_train_texts, y_train, validation_data=(x_dev_texts, y_dev),
          epochs=20, batch_size=50)

Train on 111699 samples, validate on 47872 samples
Epoch 1/20
111699/111699 [==============================] - 3805s 34ms/step - loss: 0.0868 - acc: 0.9732 - val_loss: 0.0581 - val_acc: 0.9794
Epoch 2/20
111699/111699 [==============================] - 6190s 55ms/step - loss: 0.0595 - acc: 0.9794 - val_loss: 0.0527 - val_acc: 0.9811
Epoch 3/20
111699/111699 [==============================] - 6494s 58ms/step - loss: 0.0564 - acc: 0.9802 - val_loss: 0.0518 - val_acc: 0.9813
Epoch 4/20
111699/111699 [==============================] - 2857s 26ms/step - loss: 0.0535 - acc: 0.9807 - val_loss: 0.0507 - val_acc: 0.9815
Epoch 6/20
111699/111699 [==============================] - 2870s 26ms/step - loss: 0.0523 - acc: 0.9811 - val_loss: 0.0498 - val_acc: 0.9819
Epoch 7/20
111699/111699 [==============================] - 2862s 26ms/step - loss: 0.0519 - acc: 0.9814 - val_loss: 0.0500 - val_acc: 0.9818
Epoch 8/20
111699/111699 [==============================] - 2859s 26ms/step - loss: 0.0512 - acc:

In [19]:
model_json = model.to_json()
with open(MODEL_PATH + "keras_lstm_fancy_classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_PATH + "keras_lstm_fancy_classification_model.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
model.evaluate(x_dev_texts, y_dev)

47872/47872 [==============================] - 437s 9ms/step


[0.04734471793094137, 0.9824184137869646]

In [30]:
X_dev = pickle.load(open(DATA_PATH + "X_dev.p", "rb"))
x_dev_texts = tokenizer.texts_to_sequences(X_dev['comment_text'])
x_dev_texts = pad_sequences(x_dev_texts, maxlen=max_length, padding='post')
pred = model.predict(x_dev_texts, verbose = True)

47872/47872 [==============================] - 408s 9ms/step


In [31]:
pickle.dump(pred, open("../submissions/DEV_" + model_name + ".p", "wb"))

In [25]:
model_name = 'keras_lstm_fancy_classification_model'
X_official_test = pd.read_csv("../data/raw/test.csv")
x_test_texts = tokenizer.texts_to_sequences(X_official_test['comment_text'])
x_test_texts = pad_sequences(x_test_texts, maxlen=max_length, padding='post')
pred = model.predict(x_test_texts)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission = pd.read_csv('../submissions/sample_submission.csv')
sample_submission[list_classes] = pred
sample_submission.to_csv("../submissions/" + model_name + ".csv", index=False)